<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/LLM_Chatbot_for_Beekeeping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **RAG based Beekeeping Chatbot for Indian Beekeepers**

In [ ]:
!pip install -qU \
     transformers \
     datasets \
     langchain \
     huggingface_hub \
     tiktoken \
     faiss-gpu \
     python-dotenv \
     accelerate \
     einops \
     bitsandbytes \
     unstructured unstructured[pdf] \
     pypdf \
     rapidocr-onnxruntime
!pip install -qU sentence_transformers


In [ ]:
import os
from dotenv import load_dotenv
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from langchain_community.document_loaders import PDFPlumberLoader

In [ ]:
loader = PDFPlumberLoader("/content/drive/MyDrive/Floral Calendar.pdf")

In [ ]:
document = loader.load()

In [ ]:
print(len(document))

420


In [ ]:
document[70]

Document(page_content='Guntur District\nJan Feb Mar Apr May June July Aug Sep Oct Nov Dec\nPlant Name\nP N P N P N P N P N P N P N P N P N P N P N P N\nAnacardium occidentale(Geedimamidi)\nBrassica(Avalu)\nCajanus cajan (Kandulu)\nCicer arietinum (Shanagalu)\nCitrus aurantifolia(Nimma)\nCitrus sinensis (Batavia)\nCocos nucifera (Kobbarichettu)\nCucumis sativus(Dosakaya)\nDelonix regia(Sankesulu)\nDolichos lab-lab(Anumulu)\nEmblica officinalis (Vusiri)\nEucalyptus spp.(Nilgiri)\nHygrophila auriculata()\nLuffa spp.(Beera)\nMoringa oleifera(Mulaga)\nOryza sativa (Vari)\nPeltophorum pterocarpum(Konda chinta)\nPennesetum typhoides (Sajja)\nPhoenix sylvestris(Eetha)\nRecinus communis(Amudalu)\nSapindus emarginatus(Kunkudu)\nSorghum vulgare (Jonna)\nTamarindus indica (Chintachettu)\nVigna mungo (Minumulu)\nVigna radiata(Pesalu)\nVigna trilobata(Pillipesara)\nZea mays (Mokkajhona)\nSesamum indicum (Nuvvulu)\nBorassus flabellifer(Tadi)\nPsidium guajava (Jama)\nZizyphus spp. (Reghu, pariki,Gotti

In [ ]:
document[70].page_content

'Guntur District\nJan Feb Mar Apr May June July Aug Sep Oct Nov Dec\nPlant Name\nP N P N P N P N P N P N P N P N P N P N P N P N\nAnacardium occidentale(Geedimamidi)\nBrassica(Avalu)\nCajanus cajan (Kandulu)\nCicer arietinum (Shanagalu)\nCitrus aurantifolia(Nimma)\nCitrus sinensis (Batavia)\nCocos nucifera (Kobbarichettu)\nCucumis sativus(Dosakaya)\nDelonix regia(Sankesulu)\nDolichos lab-lab(Anumulu)\nEmblica officinalis (Vusiri)\nEucalyptus spp.(Nilgiri)\nHygrophila auriculata()\nLuffa spp.(Beera)\nMoringa oleifera(Mulaga)\nOryza sativa (Vari)\nPeltophorum pterocarpum(Konda chinta)\nPennesetum typhoides (Sajja)\nPhoenix sylvestris(Eetha)\nRecinus communis(Amudalu)\nSapindus emarginatus(Kunkudu)\nSorghum vulgare (Jonna)\nTamarindus indica (Chintachettu)\nVigna mungo (Minumulu)\nVigna radiata(Pesalu)\nVigna trilobata(Pillipesara)\nZea mays (Mokkajhona)\nSesamum indicum (Nuvvulu)\nBorassus flabellifer(Tadi)\nPsidium guajava (Jama)\nZizyphus spp. (Reghu, pariki,Gotti)\nMajor Pollen Minor 

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=30,
    add_start_index=True,
    strip_whitespace=True,
    separators=MARKDOWN_SEPARATORS,
)

In [ ]:
splits = text_splitter.split_documents(document)

In [ ]:
splits[30].page_content

'Jan Feb Mar Apr May June July Aug Sep Oct Nov Dec\nDistrict\nP N P N P N P N P N P N P N P N P N P N P N P N\nLATUR\nतुवर\nCajanus cajan ( )\n1 1 1 1\nकुसुम\nCarthamus tinctorious ( )\n1 1 1 1\nचना\nCicer arietinum ( )\n1 1 1 1\nका(cid:426)ी िनबू\nCitrus aurantifolia ( )\n1 2 1 2 1 2 1 2\nस(cid:361)ा\nCitrus reticulata ( )\n1 2\nमोसबी\nCitrus sinensis ( )\n1 2\nआवला\nEmblica officinalis ( )\n2 1 2 1\nिनलिगरी\nEucalyptus spp. ( )\n2 2 2 2\nरामतील\nGuizotia abyssinica ( )\n2 2 2 2\nसुयफुल\nHelianthus annuus ( )\n2 2 2 2 2 2\nशेवगा\nMoringa oleifera ( )\n2 2 2 2\nकेला\nMusa paradisica ( )\n1 1 1 1\nचावल\nOryza sativa ( )\n1 0 1 0\nअद(cid:304)(cid:566)\nParkinsonia aculeata ( )\n2 1 2 1 2 1\nबाजरा\nPennesetum typhoides ( )\n1 0 1 0\nिशदी\nPhoenix sylvestris ( )\n2 0 2 0\nवेडी बाभूळ\nProsopis juliflora ( )\n1 2 1 2 1 2\nअम(cid:349)द\nPsidium guvajava ( )\n2 1 2 1\nकडधा(cid:586)\nPulses ( )\n1 1 1 1\nतील\nSesamum indicum ( )\n2 2 2 2\n(cid:476)ार\nSorghum vulgare ( )\n2 1 2 1 2 1 2 1\nमुग\n

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs
)

AttributeError: module 'numpy.linalg._umath_linalg' has no attribute '_ilp64'

In [ ]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(documents, embeddings)

In [ ]:
def get_similar_docs(query, k=2, score=False):
    return vectorstore.similarity_search_with_score(query, k=k) if score else vectorstore.similarity_search(query, k=k)

query = "What procedures are outlined for the restoration of data in the VLR after a failure?"
similar_docs = get_similar_docs(query)

print(similar_docs)